# Experiments

Running this notebook reproduces the experiments from the paper where we examine the effect of the value of $\beta$.

In [12]:
import math
import gudhi
import numpy as np
from miniball import Miniball
import timeit
import functools
import warnings
warnings.filterwarnings('ignore')

from core import core_alpha, sqrt_persistence, persistence_intervals_in_dimension
from datasets import sample_circle, sample_rectangle, sample_torus, sample_cube, sample_flat_torus, sample_sphere

## Helper Functions

Compute alpha-core persistent homology from a point cloud $X$ for multiple values of $s_\text{max}$.

In [13]:
def persistence_diagrams(
    X, max_ss: list[int] = [1, 10, 100, 1000], 
    max_r: float | None = -1
) -> list[tuple[int, tuple[int, int]]]:
    if max_r < 0:
        max_r = 2*math.sqrt(Miniball(X).squared_radius())
    res = []
    for i, max_s in enumerate(max_ss):
        max_k = max(1, int((M + N) * max_s))
        st = core_alpha(X, max_k=max_k, max_r=max_r)
        persistence = sqrt_persistence(st)
        res.append(persistence)
    return res

### Point Cloud Dataset Generators

Functions for generating the datasets.

In [14]:
def circle_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_circle(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]    

In [15]:
def two_circles_with_noise(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    N1 = (2 * N) // 3
    N2 = N // 3 
    
    Z1 = sample_circle(N1, rng, r=1, std=sigma)
    Z2 = sample_circle(N2, rng, r=0.5, std=sigma)
    Z = np.r_[Z1, Z2]
    
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z1, Z2, Y]

In [16]:
def embedded_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_torus(N, rng, a=1, b=3, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [17]:
def sphere(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_sphere(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

In [18]:
def clifford_torus(M, N, sigma, rng=None, seed=0):
    if rng is None:
        rng = np.random.default_rng(seed=seed)
    Z = sample_flat_torus(N, rng, std=sigma)
    upper_right_corner = np.maximum(np.max(Z, axis=0), -np.min(Z, axis=0))
    Y = sample_rectangle(M, rng, lower_left_corner=-upper_right_corner, upper_right_corner=upper_right_corner)
    return np.r_[Z, Y]

### Bottleneck Distances

The following function compute bottleneck distances between the alpha-core persistence and the ground truth diagrams for a given point cloud dataset generator.

In [28]:
def bottleneck_distance_beta_experiment(
    betas, M, N, max_s=0.001, sigma=0.07, max_r=-1, 
    point_generator=circle_with_noise, seed=0):
    rng = np.random.default_rng(seed=seed)
    X = point_generator(0, 5, sigma=0, rng=rng)
    dimensions = range(X.shape[1])
    res = {dim: [] for dim in dimensions}
    
    X = point_generator(0, M + N, sigma=0, rng=rng)
    st_ideal = core_alpha(X, max_k=1, max_r=max_r, beta=1)
    persistence_ideal = sqrt_persistence(st_ideal) 
    for beta in betas:
        print(f"beta={beta}")
        X = point_generator(M, N, sigma, rng=rng)
        if max_r is not None:
            max_r = 2*math.sqrt(Miniball(X).squared_radius())
        max_k = max(1, int((M + N) * max_s))
        print(f"\tmax_s={max_s} (max_k = {max_k})")
        st = core_alpha(X, max_k=max_k, max_r=max_r, beta=beta)
        persistence = sqrt_persistence(st)
        for dim in dimensions:
            res[dim].append(
                gudhi.bottleneck_distance(
                    persistence_intervals_in_dimension(persistence, dim),
                    persistence_intervals_in_dimension(persistence_ideal, dim)))
    return res
        
        

Helper function for printing the results.

In [53]:
def formatted_bottleneck_results(bottleneck_distances, betas, M, N, max_s):
    res = [f"betas={betas}, N={N}, M={M}, max_s={max_s}"]
    for dim in bottleneck_distances.keys():
        res.append("\\addplot plot coordinates {" + " ".join([f"({beta}, {dist:.5f})" for beta, dist in zip(betas, bottleneck_distances[dim])]) + f"}};\\addlegendentry{{Dim {dim}}}")
    return res

Function for running the experiments with a list of different point cloud dataset generators and a list of $s_\text{max}$.

In [54]:
def run_experiments_beta(
    point_generators,
    betas,
    M,
    N,
    max_s,
    names=None,
    sigma=0.07,
    seed=0,
    max_r=-1,
):
    res = []
    for idx, generator in enumerate(point_generators):
        if names is not None:
            name = names[idx]
        else:
            name = 'Unknown'
        print(f"Running experiments for {name}")
        bottleneck_dists = bottleneck_distance_beta_experiment(
            betas=betas, M=M, N=N, max_s=max_s, point_generator=generator, sigma=sigma, seed=seed, max_r=max_r)
        res.append([name] +
                   formatted_bottleneck_results(bottleneck_dists, betas, M, N, max_s))
    return res

## Run Experiments
### Experiment Settings

In [65]:
betas = [2 ** i for i in range(-3, 4)]
M = 1000
N = 10000
max_s = 0.01
sigma = 0.07
seed = 0
point_generators = [circle_with_noise, two_circles_with_noise, sphere, embedded_torus, clifford_torus]
names = ["Circle", "Circles", "Sphere", "Torus 1", "Torus 2"]

### Persistence along a line

Run experiments for computing alpha-core persistence along a line.

In [66]:
%%time
result = run_experiments_beta(
    point_generators,
    betas=betas,
    M = M,
    N = N,
    max_s = max_s,
    sigma=sigma,
    seed=seed,
    names = names)

Running experiments for Circle
beta=0.125
	max_s=0.01 (max_k = 110)
beta=0.25
	max_s=0.01 (max_k = 110)
beta=0.5
	max_s=0.01 (max_k = 110)
beta=1
	max_s=0.01 (max_k = 110)
beta=2
	max_s=0.01 (max_k = 110)
beta=4
	max_s=0.01 (max_k = 110)
beta=8
	max_s=0.01 (max_k = 110)
Running experiments for Circles
beta=0.125
	max_s=0.01 (max_k = 110)
beta=0.25
	max_s=0.01 (max_k = 110)
beta=0.5
	max_s=0.01 (max_k = 110)
beta=1
	max_s=0.01 (max_k = 110)
beta=2
	max_s=0.01 (max_k = 110)
beta=4
	max_s=0.01 (max_k = 110)
beta=8
	max_s=0.01 (max_k = 110)
Running experiments for Sphere
beta=0.125
	max_s=0.01 (max_k = 110)
beta=0.25
	max_s=0.01 (max_k = 110)
beta=0.5
	max_s=0.01 (max_k = 110)
beta=1
	max_s=0.01 (max_k = 110)
beta=2
	max_s=0.01 (max_k = 110)
beta=4
	max_s=0.01 (max_k = 110)
beta=8
	max_s=0.01 (max_k = 110)
Running experiments for Torus 1
beta=0.125
	max_s=0.01 (max_k = 110)
beta=0.25
	max_s=0.01 (max_k = 110)
beta=0.5
	max_s=0.01 (max_k = 110)
beta=1
	max_s=0.01 (max_k = 110)
beta=2
	max_s

Print the results formatted for use with pgfplots.

In [67]:
print('\n'.join(['\n'.join(x) for x in result]))

Circle
betas=[0.125, 0.25, 0.5, 1, 2, 4, 8], N=10000, M=1000, max_s=0.01
\addplot plot coordinates {(0.125, 0.01907) (0.25, 0.02751) (0.5, 0.03793) (1, 0.05549) (2, 0.07676) (4, 0.11115) (8, 0.15454)};\addlegendentry{Dim 0}
\addplot plot coordinates {(0.125, 0.49824) (0.25, 0.49824) (0.5, 0.49824) (1, 0.49824) (2, 0.37360) (4, 0.14312) (8, 0.30711)};\addlegendentry{Dim 1}
Circles
betas=[0.125, 0.25, 0.5, 1, 2, 4, 8], N=10000, M=1000, max_s=0.01
\addplot plot coordinates {(0.125, 0.12500) (0.25, 0.12500) (0.5, 0.12500) (1, 0.08623) (2, 0.09707) (4, 0.13024) (8, 0.19088)};\addlegendentry{Dim 0}
\addplot plot coordinates {(0.125, 0.24845) (0.25, 0.24845) (0.5, 0.24845) (1, 0.16802) (2, 0.12847) (4, 0.17059) (8, 0.35133)};\addlegendentry{Dim 1}
Sphere
betas=[0.125, 0.25, 0.5, 1, 2, 4, 8], N=10000, M=1000, max_s=0.01
\addplot plot coordinates {(0.125, 0.06591) (0.25, 0.09104) (0.5, 0.12952) (1, 0.17775) (2, 0.26132) (4, 0.37254) (8, 0.50437)};\addlegendentry{Dim 0}
\addplot plot coordinates